In [534]:
#pip install chromadb

In [535]:
import os
import pandas as pd
from datetime import datetime
import numpy as np
import nltk
import torch
from io import BytesIO

from tqdm.notebook import tqdm
import chromadb

import pandas as pd
import torch

from transformers import BertTokenizer, BertModel
from transformers import AutoModel, AutoTokenizer



# Ensure that the Punkt Tokenizer Models are downloaded
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\johnny\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [536]:
#download_dir = 'D:\\downloads'
download_dir = 'D:\\downloads\\amazon_customer_reviews'
print(download_dir)

D:\downloads\amazon_customer_reviews


In [537]:
#df_pickle_paragraphs_filename = os.path.join(download_dir,"amazon_reviews_pickle_paragraphs_20240129.pkl")
df_pickle_paragraphs_filename = os.path.join(download_dir,"amazon_reviews_pickle_paragraphs_20240219.pkl")
df_paragraph = pd.read_pickle(df_pickle_paragraphs_filename)

In [538]:
df_paragraph.columns

Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Paragraph',
       'Paragraph_vector', 'Summary_vector'],
      dtype='object')

In [539]:
df_paragraph.shape

(568427, 12)

In [540]:
df_paragraph.iloc[5]

Id                                                                        6
ProductId                                                        B006K2ZZ7K
UserId                                                        ADT0SRK1MGOEU
ProfileName                                                  Twoapennything
HelpfulnessNumerator                                                      0
HelpfulnessDenominator                                                    0
Score                                                                     4
Time                                                             1342051200
Summary                                                          Nice Taffy
Paragraph                 I got a wild hair for taffy and ordered this f...
Paragraph_vector          [-0.08328723, 0.08941264, 0.39326283, 0.106011...
Summary_vector            [0.45653412, -0.095085524, 0.59296936, -0.1120...
Name: 5, dtype: object

In [541]:
#df_pickle_sentences_filename = os.path.join(download_dir,"amazon_reviews_pickle_sentences_20240129.pkl")
df_pickle_sentences_filename = os.path.join(download_dir,"amazon_reviews_pickle_sentences_20240219.pkl")

df_sentences = pd.read_pickle(df_pickle_sentences_filename)

In [542]:
df_sentences.columns

Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'P_index',
       'S_sentence_number', 'Sentence', 'Summary_vector', 'Sentence_vector'],
      dtype='object')

In [543]:
df_sentences.shape

(2832752, 14)

In [544]:
df_sentences.iloc[5]

Id                                                                        3
ProductId                                                        B000LQOCH0
UserId                                                        ABXLMWJIXXAIN
ProfileName                                 Natalia Corres "Natalia Corres"
HelpfulnessNumerator                                                      1
HelpfulnessDenominator                                                    1
Score                                                                     4
Time                                                             1219017600
Summary                                               "Delight" says it all
P_index                                                                 P_2
S_sentence_number                                                       S_1
Sentence                  This is a confection that has been around a fe...
Summary_vector            [0.078089125, 0.12389954, 0.563005, 0.06509149...
Sentence_vec

In [545]:
df_sentences.iloc[5]['Summary_vector'].shape

(768,)

In [546]:
df_sentences.iloc[5]['Summary_vector'][0].shape

()

In [547]:
type(df_sentences.iloc[5]['Summary_vector'][0])

numpy.float32

In [548]:
df_sentences.columns

Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'P_index',
       'S_sentence_number', 'Sentence', 'Summary_vector', 'Sentence_vector'],
      dtype='object')

In [549]:
df = df_sentences.iloc[0:2].copy()

In [550]:
import faiss

def create_faiss_index_old(vectors):
    dimension = vectors.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(vectors)
    return index

In [551]:
def create_faiss_index_old(vectors):
    # Example setup for FAISS index - this is conceptual
    index = faiss.IndexFlatL2(vectors.shape[1])
    # Example of batch addition with tqdm progress bar
    for i in tqdm(range(0, vectors.shape[0], 1000), desc="Adding vectors to FAISS index"):
        index.add(vectors[i:i+1000])
    return index


In [552]:
len(df_sentences.iloc[0]['Summary_vector'])

768

In [553]:
len(df_sentences.iloc[167494]['Summary_vector'])

768

In [554]:
df_paragraph.columns

Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Paragraph',
       'Paragraph_vector', 'Summary_vector'],
      dtype='object')

In [555]:
df_sentences.iloc[167494]

Id                                                                    33960
ProductId                                                        B00412W76S
UserId                                                       A20X23WJLV4R5X
ProfileName                                                kittykatbookworm
HelpfulnessNumerator                                                     18
HelpfulnessDenominator                                                   18
Score                                                                     5
Time                                                             1251849600
Summary                                              tasty Maybe beneficial
P_index                                                             P_33959
S_sentence_number                                                       S_1
Sentence                  My midwife recommended this product, and I dra...
Summary_vector            [0.034660093, -0.20093988, -0.12915072, 0.1708...
Sentence_vec

In [556]:
# this deletes rows where paragraph or summary are nan (empty)
df_paragraph = df_paragraph.dropna(subset=['Paragraph', 'Summary'])

In [557]:
# this deletes rows where the summary or sentence is a nan (empty)
print(f"Original df_sentences rows: {len(df_sentences)}") # how many rows are there to start
# Drop rows where 'Summary' is NaN
df_sentences= df_sentences.dropna(subset=['Summary'])

# Optionally, also drop rows where 'Sentence' is NaN if relevant to your analysis
df_sentences = df_sentences.dropna(subset=['Sentence'])

print(f"Cleaned df_sentences rows: {len(df_sentences)}") # how many rows are there now


Original df_sentences rows: 2832752
Cleaned df_sentences rows: 2832752


In [558]:
from tqdm.auto import tqdm
import faiss
import numpy as np
import pandas as pd

def create_faiss_index(vectors, n_clusters):
    dimension = vectors.shape[1]  # Dimension of the vectors
    kmeans = faiss.Kmeans(dimension, n_clusters, niter=20, verbose=True, spherical=False)
    kmeans.train(vectors)
    return kmeans

def cluster_vectors_and_save(df, vector_column, n_clusters, cluster_col_name, distance_col_name):
    # this takes in a dataframe, the number of clusters that you want to make, and 2 column names that you want to add to the dataframe
    # the columns are to hold the cluster that the row is in
    # Filter out NaNs and prepare the vectors
    filtered_df = df.dropna(subset=[vector_column])
    vectors = np.vstack(filtered_df[vector_column].apply(np.array).to_list())

    # Initialize the progress bar
    tqdm.pandas(desc=f"Clustering {vector_column}")
    
    # Create a FAISS KMeans index for clustering
    print(f"Training KMeans for {vector_column}...")
    kmeans = create_faiss_index(vectors, n_clusters)

    # Assign vectors to the nearest cluster and capture distances
    print(f"Assigning clusters for {vector_column}...")
    distances, cluster_assignments = kmeans.index.search(vectors, 1)

    # Since we filtered NaNs, align results with the original DataFrame indices
    valid_indices = filtered_df.index

    # Update the original DataFrame. This makes 2 new columns in the original dataframe
    df.loc[valid_indices, cluster_col_name] = cluster_assignments.flatten()
    df.loc[valid_indices, distance_col_name] = distances.flatten()

    return df, kmeans

# Example usage with a specified number of clusters
n_clusters = 100

# Assuming df_sentences and df_paragraph are your DataFrames and they are properly initialized
#df_sentences, index_summary  = cluster_vectors_and_save(df_sentences, 'Summary_vector', n_clusters, 'Summary_Cluster', 'Summary_Distance')
df_sentences, cluster_index_sentence  = cluster_vectors_and_save(df_sentences, 'Sentence_vector', n_clusters, 'Sentence_Cluster', 'Sentence_Distance')
df_paragraph, cluster_index_paragraph  = cluster_vectors_and_save(df_paragraph, 'Paragraph_vector', n_clusters, 'Paragraph_Cluster', 'Paragraph_Distance')
df_paragraph, cluster_index_summary  = cluster_vectors_and_save(df_paragraph, 'Summary_vector', n_clusters, 'Summary_Cluster', 'Summary_Distance')

Training KMeans for Sentence_vector...
Assigning clusters for Sentence_vector...
Training KMeans for Paragraph_vector...
Assigning clusters for Paragraph_vector...
Training KMeans for Summary_vector...
Assigning clusters for Summary_vector...


In [559]:
df_paragraph.columns

Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Paragraph',
       'Paragraph_vector', 'Summary_vector', 'Paragraph_Cluster',
       'Paragraph_Distance', 'Summary_Cluster', 'Summary_Distance'],
      dtype='object')

In [560]:
df_sentences.iloc[0]

Id                                                                        1
ProductId                                                        B001E4KFG0
UserId                                                       A3SGXH7AUHU8GW
ProfileName                                                      delmartian
HelpfulnessNumerator                                                      1
HelpfulnessDenominator                                                    1
Score                                                                     5
Time                                                             1303862400
Summary                                               Good Quality Dog Food
P_index                                                                 P_0
S_sentence_number                                                       S_1
Sentence                  I have bought several of the Vitality canned d...
Summary_vector            [0.10738717, 0.015432298, -0.28650856, 0.00476...
Sentence_vec

# good stuff

In [561]:

def create_faiss_index_old(vectors):
    dimension = vectors.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(vectors)
    return index

In [562]:
import time

# Start time
start_time = time.perf_counter()

In [563]:
#stop

# sentence

In [564]:
# This is an important processing step that is required for processing the sentence vector into faiss indexes
# sentence_vector
Sentence_vector_list = np.array(df_sentences['Sentence_vector'].tolist()).astype('float32')

In [565]:
index_sentence = create_faiss_index_old(Sentence_vector_list)

In [625]:
index_sentence_filename = os.path.join(download_dir,"amazon_reviews_pickle_sentences_similarity_20240219_index_sentence.faiss")

faiss.write_index(index_sentence, index_sentence_filename)

In [566]:
# this is expected to take around 36 seconds
start_time = time.perf_counter()
query_vectors = Sentence_vector_list[:1000] 
D, I = index_sentence.search(query_vectors, 100)
print("Indices of nearest neighbors: ", I[0])
print("Distances: ", D[0])
end_time = time.perf_counter()
duration = end_time - start_time
print(f"The block of code took {duration:.6f} seconds to execute.")
projected_duration = duration * Sentence_vector_list.shape[0] / 1000
projected_minutes = projected_duration / 60
projected_hours = projected_minutes / 60
print(f"the projected duration of the entire vector is {projected_duration:.2f} seconds - {projected_minutes:.2f} minutes - {projected_hours:.2f} hours to execute.")

Indices of nearest neighbors:  [      0 1648290 1457255 1105728   11931  194732 2270483   27323 1564854
  500702  214940 2357430 1699082  934162 2413615  490504   24406 2235628
  986503 1004141 1834620 2704142  171935 2681032 2609400 1236060 1563049
 2659349 1010907 1341792 1096364  910505  417076  484536 1390625 1976488
 1062097  632549 2039959 2765030 1923011  856326 1399801 2804320 1364129
 1058201  250729 2539702  479771 2437130  642371 2238572 1378245 2793435
 2581837 1414707 1662265  387037 2131983 2468596  476414 2511871  962070
 1892188 2266993  539868 1811553 1072813 2379517  563650  866931  640678
  822924  654852 2308226 2392336 1480170 2636107 2708434 1418075 2708595
  886490 1903565 2119827  898131  590794  614089  794040 2378514  543504
  619889 2274346  505114 1374299 1832213 2391859 2758004 2343687 1565404
  424864]
Distances:  [ 0.       12.972076 14.787857 15.135315 16.024734 16.294922 16.75058
 17.21521  17.378815 17.460266 17.718002 17.730347 17.816193 18.167877
 18

In [606]:
df_sentences.columns

Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'P_index',
       'S_sentence_number', 'Sentence', 'Summary_vector', 'Sentence_vector',
       'Sentence_Cluster', 'Sentence_Distance', 'Sentence_similarity_distance',
       'Sentence_similarity_index'],
      dtype='object')

In [567]:
# this is expected to take around 20.55 hours
start_time = time.perf_counter()
D, I = index_sentence.search(Sentence_vector_list, 100)
# Convert D and I to lists and add them as new columns
df_sentences['Sentence_similarity_distance'] = list(D)
df_sentences['Sentence_similarity_index'] = list(I)
end_time = time.perf_counter()
duration = end_time - start_time
print(f"The block of code took {duration:.6f} seconds to execute.")

The block of code took 59332.142764 seconds to execute.


In [571]:
projected_duration = duration * Paragraph_vector_list.shape[0] / 1000
projected_minutes = projected_duration / 60
projected_hours = projected_minutes / 60
print(f"the projected duration of the entire vector is {projected_duration:.2f} seconds - {projected_minutes:.2f} minutes - {projected_hours:.2f} hours to execute.")

the projected duration of the entire vector is 33725991.91 seconds - 562099.87 minutes - 9368.33 hours to execute.


In [568]:
# Convert D and I to lists and add them as new columns
df_sentences['Sentence_similarity_distance'] = list(D)
df_sentences['Sentence_similarity_index'] = list(I)

In [570]:
df_pickle_filename = os.path.join(download_dir,"amazon_reviews_pickle_sentences_similarity_20240219.pkl")
df_sentences.to_pickle(df_pickle_filename)
print(df_pickle_filename)

D:\downloads\amazon_customer_reviews\amazon_reviews_pickle_sentences_similarity_20240219.pkl


# Summary

In [572]:
# summary_vector
Summary_vector_list = np.array(df_paragraph['Summary_vector'].tolist()).astype('float32')

In [573]:
index_summary = create_faiss_index_old(Summary_vector_list)


In [626]:
index_summary_filename = os.path.join(download_dir,"amazon_reviews_pickle_sentences_similarity_20240219_index_summary.faiss")

faiss.write_index(index_summary, index_summary_filename)

In [574]:
# this is expected to take around 36 seconds
start_time = time.perf_counter()
query_vectors = Summary_vector_list[:1000] 
D, I = index_summary.search(query_vectors, 100)
print("Indices of nearest neighbors: ", I[0])
print("Distances: ", D[0])
end_time = time.perf_counter()
duration = end_time - start_time
print(f"The block of code took {duration:.6f} seconds to execute.")
projected_duration = duration * Paragraph_vector_list.shape[0] / 1000
projected_minutes = projected_duration / 60
projected_hours = projected_minutes / 60
print(f"the projected duration of the entire vector is {projected_duration:.2f} seconds - {projected_minutes:.2f} minutes - {projected_hours:.2f} hours to execute.")

Indices of nearest neighbors:  [382666      0 399359 489673  53168 221484  24587 144559  58892 179267
 489807 179559 237222 249997 308826 156565 130790 256872 257297 140106
 246845 123858  53993 240237 546826 280248  80214 292611 197335 481025
 425084 357520 272799 471405 408685 399362 389326 457292 435098  24730
  43956 534734 431616  17680 546814   4741 478079 351362  13702  93694
  53981 390602 221514 112837 390672 123846 240116  58922 534102 113382
 144589 506154 273688 234223 489837  24617 280236 460904 471580 292599
 533922 328765  59685 375852 272206  59694  46024 173265  73176 513559
  80202 433171 179297 202140   1587 431634 429104 402906  89779 447699
 173250     95 457375 444982 216132 240075 221538 216046 268057 255458]
Distances:  [ 0.         0.         3.1913757  3.4827728  3.4827728  4.535492
  4.535492   4.535492   4.535492   4.535492   4.535492   4.535492
  4.535492   4.6536713  4.6536713  4.6536713  4.6536713  4.6536713
  4.6536713  4.6536713  4.6536713  4.732071   4

In [575]:
# this is expected to take around 36 seconds
start_time = time.perf_counter()
query_vectors = Summary_vector_list
D, I = index_summary.search(query_vectors, 100)
print("Indices of nearest neighbors: ", I[0])
print("Distances: ", D[0])
end_time = time.perf_counter()
duration = end_time - start_time
print(f"The block of code took {duration:.6f} seconds to execute.")
Seconds_duration = duration 
projected_minutes = Seconds_duration / 60
projected_hours = projected_minutes / 60
print(f"The block of code took {Seconds_duration:.2f} seconds - {projected_minutes:.2f} minutes - {projected_hours:.2f} hours to execute.")

Indices of nearest neighbors:  [382666      0 399359 489673  53168 221484  24587 144559  58892 179267
 489807 179559 237222 249997 308826 156565 130790 256872 257297 140106
 246845 123858  53993 240237 546826 280248  80214 292611 197335 481025
 425084 357520 272799 471405 408685 399362 389326 457292 435098  24730
  43956 534734 431616  17680 546814   4741 478079 351362  13702  93694
  53981 390602 221514 112837 390672 123846 240116  58922 534102 113382
 144589 506154 273688 234223 489837  24617 280236 460904 471580 292599
 533922 328765  59685 375852 272206  59694  46024 173265  73176 513559
  80202 433171 179297 202140   1587 431634 429104 402906  89779 447699
 173250     95 457375 444982 216132 240075 221538 216046 268057 255458]
Distances:  [ 0.         0.         3.1913757  3.4827728  3.4827728  4.535492
  4.535492   4.535492   4.535492   4.535492   4.535492   4.535492
  4.535492   4.6536713  4.6536713  4.6536713  4.6536713  4.6536713
  4.6536713  4.6536713  4.6536713  4.732071   4

In [576]:
# Convert D and I to lists and add them as new columns
df_paragraph['Summary_similarity_distance'] = list(D)
df_paragraph['Summary_similarity_index'] = list(I)

In [621]:
df_paragraph.iloc[0]

Id                                                                               1
ProductId                                                               B001E4KFG0
UserId                                                              A3SGXH7AUHU8GW
ProfileName                                                             delmartian
HelpfulnessNumerator                                                             1
HelpfulnessDenominator                                                           1
Score                                                                            5
Time                                                                    1303862400
Summary                                                      Good Quality Dog Food
Paragraph                        I have bought several of the Vitality canned d...
Paragraph_vector                 [-0.12606543, 0.062131397, 0.0551221, -0.00242...
Summary_vector                   [0.10738717, 0.015432298, -0.28650856, 0.00476...
Para

# paragraph

In [577]:
# summary_vector
Paragraph_vector_list = np.array(df_paragraph['Paragraph_vector'].tolist()).astype('float32')
print(Paragraph_vector_list.shape[0], " length of paragraph_vector_list")

568427  length of paragraph_vector_list


In [578]:
index_paragraph = create_faiss_index_old(Paragraph_vector_list)

In [627]:
index_paragraph_filename = os.path.join(download_dir,"amazon_reviews_pickle_sentences_similarity_20240219_index_paragraph.faiss")

faiss.write_index(index_paragraph, index_paragraph_filename)

In [579]:
# this is expected to take around 36 seconds
start_time = time.perf_counter()
query_vectors = Paragraph_vector_list[:1000] 
D, I = index_paragraph.search(query_vectors, 100)
print("Indices of nearest neighbors: ", I[0])
print("Distances: ", D[0])
end_time = time.perf_counter()
duration = end_time - start_time
print(f"The block of code took {duration:.6f} seconds to execute.")
projected_duration = duration * Paragraph_vector_list.shape[0] / 1000
projected_minutes = projected_duration / 60
projected_hours = projected_minutes / 60
print(f"the projected duration of the entire vector is {projected_duration:.2f} seconds - {projected_minutes:.2f} minutes - {projected_hours:.2f} hours to execute.")

Indices of nearest neighbors:  [     0 357080 399351 118925  87038 174270  77939 524385 506600 509370
 238877 198732 407622  77238 322662 112837  89760 177785 208743 119607
 242786 341089 339590 276987 285705 508415  47652 148269 139618 539207
 184475  20291 116865 442312 356994 171393 115526 277988 146493 410231
 343243 486079 191541 437078 211242 444976 509258 399177  50954 383008
 306378 169618 414238 505381 110205 271600  18318 195419   8264 447699
 560668  41860 253063 359988 146781  80964 253565 512306 162591 273712
 322274 411187  84855 525083 386033 241405 295356 439358 160889 510108
 379110  54435 555674 210219 272272 514112 169374 475930 325901 240225
  14205 309251 213124 426659 292697  83226 160240 401848 300072 315123]
Distances:  [ 0.         9.213089   9.347366   9.460304   9.700714   9.70076
  9.70076    9.70076    9.70076    9.70076    9.70076    9.70076
  9.70076    9.70076    9.701889  10.113693  10.255371  10.263527
 10.2729645 10.301964  10.332123  10.355682  10.42

In [580]:
# this is expected to take around 48.92 minutes
start_time = time.perf_counter()
query_vectors = Paragraph_vector_list
D, I = index_paragraph.search(query_vectors, 100)
print("Indices of nearest neighbors: ", I[0])
print("Distances: ", D[0])
end_time = time.perf_counter()
duration = end_time - start_time
print(f"The block of code took {duration:.6f} seconds to execute.")

Indices of nearest neighbors:  [     0 357080 399351 118925  87038 174270  77939 524385 506600 509370
 238877 198732 407622  77238 322662 112837  89760 177785 208743 119607
 242786 341089 339590 276987 285705 508415  47652 148269 139618 539207
 184475  20291 116865 442312 356994 171393 115526 277988 146493 410231
 343243 486079 191541 437078 211242 444976 509258 399177  50954 383008
 306378 169618 414238 505381 110205 271600  18318 195419   8264 447699
 560668  41860 253063 359988 146781  80964 253565 512306 162591 273712
 322274 411187  84855 525083 386033 241405 295356 439358 160889 510108
 379110  54435 555674 210219 272272 514112 169374 475930 325901 240225
  14205 309251 213124 426659 292697  83226 160240 401848 300072 315123]
Distances:  [ 0.         9.213089   9.347366   9.460304   9.700714   9.70076
  9.70076    9.70076    9.70076    9.70076    9.70076    9.70076
  9.70076    9.70076    9.701889  10.113693  10.255371  10.263527
 10.2729645 10.301964  10.332123  10.355682  10.42

In [585]:
len(query_vectors)

568427

In [581]:
# Convert D and I to lists and add them as new columns
df_paragraph['Paragraph_similarity_distance'] = list(D)
df_paragraph['Paragraph_similarity_index'] = list(I)

In [582]:
df_pickle_filename = os.path.join(download_dir,"amazon_reviews_pickle_paragraphs_similarity_20240219.pkl")
df_paragraph.to_pickle(df_pickle_filename)
print(df_pickle_filename)

D:\downloads\amazon_customer_reviews\amazon_reviews_pickle_paragraphs_similarity_20240219.pkl


In [628]:
columns_to_drop = ['Paragraph_vector', 'Summary_vector']
df_paragraph_reduced = df_paragraph.drop(columns=columns_to_drop)

In [629]:
df_pickle_reduced_filename = os.path.join(download_dir,"amazon_reviews_pickle_paragraphs_similarity_reduced_20240219.pkl")
df_paragraph_reduced.to_pickle(df_pickle_reduced_filename)
print(df_pickle_reduced_filename)

D:\downloads\amazon_customer_reviews\amazon_reviews_pickle_paragraphs_similarity_reduced_20240219.pkl


In [586]:
df_paragraph.columns


Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Paragraph',
       'Paragraph_vector', 'Summary_vector', 'Paragraph_Cluster',
       'Paragraph_Distance', 'Summary_Cluster', 'Summary_Distance',
       'Summary_similarity_distance', 'Summary_similarity_index',
       'Paragraph_similarity_distance', 'Paragraph_similarity_index'],
      dtype='object')

In [604]:
df_paragraph.iloc[0]['Paragraph']

'I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.'

In [597]:
df_paragraph.iloc[0]['Summary_similarity_distance']

array([ 0.       ,  0.       ,  3.1913757,  3.4827728,  3.4827728,
        4.535492 ,  4.535492 ,  4.535492 ,  4.535492 ,  4.535492 ,
        4.535492 ,  4.535492 ,  4.535492 ,  4.6536713,  4.6536713,
        4.6536713,  4.6536713,  4.6536713,  4.6536713,  4.6536713,
        4.6536713,  4.732071 ,  4.732071 ,  4.732071 ,  4.732071 ,
        4.732071 ,  4.732071 ,  4.732086 ,  7.308243 ,  8.115768 ,
        8.115768 ,  8.115784 ,  8.115784 ,  8.28006  ,  8.28006  ,
        8.509232 ,  9.400925 ,  9.421875 ,  9.421875 ,  9.421875 ,
        9.955185 ,  9.994202 , 10.049683 , 10.142471 , 11.327225 ,
       11.327225 , 11.327225 , 11.327225 , 11.327225 , 11.327225 ,
       11.3272705, 11.3272705, 11.3272705, 11.3272705, 11.3272705,
       11.3272705, 11.3272705, 11.3272705, 11.3272705, 11.3272705,
       11.3272705, 11.3272705, 11.3272705, 11.3272705, 11.3272705,
       11.3272705, 11.3272705, 11.3272705, 11.3272705, 11.3272705,
       11.3272705, 11.3272705, 11.3272705, 11.3272705, 11.3272

In [602]:
df_paragraph.iloc[0]['Summary_similarity_index']

array([382666,      0, 399359, 489673,  53168, 221484,  24587, 144559,
        58892, 179267, 489807, 179559, 237222, 249997, 308826, 156565,
       130790, 256872, 257297, 140106, 246845, 123858,  53993, 240237,
       546826, 280248,  80214, 292611, 197335, 481025, 425084, 357520,
       272799, 471405, 408685, 399362, 389326, 457292, 435098,  24730,
        43956, 534734, 431616,  17680, 546814,   4741, 478079, 351362,
        13702,  93694,  53981, 390602, 221514, 112837, 390672, 123846,
       240116,  58922, 534102, 113382, 144589, 506154, 273688, 234223,
       489837,  24617, 280236, 460904, 471580, 292599, 533922, 328765,
        59685, 375852, 272206,  59694,  46024, 173265,  73176, 513559,
        80202, 433171, 179297, 202140,   1587, 431634, 429104, 402906,
        89779, 447699, 173250,     95, 457375, 444982, 216132, 240075,
       221538, 216046, 268057, 255458], dtype=int64)

In [624]:
df_paragraph['ProductId'].value_counts()

ProductId
B007JFMH8M    913
B0026RQTGE    632
B002QWHJOU    632
B002QWP89S    632
B002QWP8H0    632
             ... 
B004DSPTTM      1
B008C9QWU8      1
B007O5A6BM      1
B003Q4TZ08      1
B001LR2CU2      1
Name: count, Length: 74258, dtype: int64

In [623]:

# Assuming 'value_counts_series' is your Series object from the 'value_counts()' operation
value_counts_series = df_paragraph['ProductId'].value_counts()

# Define bins for the range of counts you're interested in. Adjust these as needed.
bins = [1,2,5, 10, 50, 100,  1000]

# Use pd.cut() to bin the value counts into the specified ranges
binned_counts = pd.cut(value_counts_series, bins, right=False)

# Count the number of product IDs in each bin
binned_counts_summary = binned_counts.value_counts().sort_index()

print(binned_counts_summary)
print(binned_counts_summary.sum())

count
[1, 2)         30408
[2, 5)         23435
[5, 10)         9796
[10, 50)        8775
[50, 100)        978
[100, 1000)      866
Name: count, dtype: int64
74258


In [615]:
#Score
def columns_bins(column_name):
    
    # Assuming 'value_counts_series' is your Series object from the 'value_counts()' operation
    value_counts_series = df_paragraph[column_name].value_counts()
    print(value_counts_series)
    # Define bins for the range of counts you're interested in. Adjust these as needed.
    bins = [1,2,5, 10, 50, 100,  1000]
    
    # Use pd.cut() to bin the value counts into the specified ranges
    binned_counts = pd.cut(value_counts_series, bins, right=False)
    
    # Count the number of product IDs in each bin
    binned_counts_summary = binned_counts.value_counts().sort_index()
    
    print(binned_counts_summary)
    return(binned_counts_summary)

In [617]:
my_bin_data = columns_bins('Score')

Score
5    363122
4     80655
1     52268
3     42638
2     29744
Name: count, dtype: int64
count
[1, 2)         0
[2, 5)         0
[5, 10)        0
[10, 50)       0
[50, 100)      0
[100, 1000)    0
Name: count, dtype: int64


In [609]:
binned_counts_summary.sum()

43850

In [619]:
import pandas as pd

# Sample DataFrame creation (replace this with your actual DataFrame)
# df = ...

# Step 1: Calculate the frequency of each ProductId
product_counts = df_paragraph['ProductId'].value_counts().reset_index()
product_counts.columns = ['ProductId', 'Count']

# Step 2: Bin these frequencies
bins = [1, 2, 5, 10, 50, 100, 1000]
product_counts['Bin'] = pd.cut(product_counts['Count'], bins, right=False)

# Merge the binned counts back to the original DataFrame to associate each row with a bin
df = df_paragraph.merge(product_counts[['ProductId', 'Bin']], on='ProductId')

# Step 3 & 4: For each ProductId, calculate the distribution of scores, then aggregate by bins
# First, group by both ProductId and Score to get the count of each score for each ProductId
score_distribution = df.groupby(['ProductId', 'Score']).size().reset_index(name='ScoreCount')

# Merge this distribution back with the bins
score_distribution = score_distribution.merge(product_counts[['ProductId', 'Bin']], on='ProductId')

# Now, aggregate this information by bin and score
final_distribution = score_distribution.groupby(['Bin', 'Score'])['ScoreCount'].sum().unstack(fill_value=0)

print(final_distribution)


Score            1     2      3      4       5
Bin                                           
[1, 2)        3215  1580   2196   3909   19508
[2, 5)        6704  3251   4104   7668   40640
[5, 10)       6233  3305   4033   8041   42314
[10, 50)     15696  8449  11509  22145  110841
[50, 100)     5672  3270   4927   9753   43464
[100, 1000)  14748  9889  15869  29139  106355


C:\Users\johnny\AppData\Local\Temp\ipykernel_4536\1979155073.py:25: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  final_distribution = score_distribution.groupby(['Bin', 'Score'])['ScoreCount'].sum().unstack(fill_value=0)


In [620]:
# Assuming 'final_distribution' is the DataFrame you've obtained from the previous steps

# Sum the rows to get subtotals for each bin
final_distribution['Subtotal'] = final_distribution.sum(axis=1)

# Display the updated DataFrame with subtotals
print(final_distribution)

# Sum the subtotals to get the grand total
grand_total = final_distribution['Subtotal'].sum()

print(f"Grand Total: {grand_total}")


Score            1     2      3      4       5  Subtotal
Bin                                                     
[1, 2)        3215  1580   2196   3909   19508     30408
[2, 5)        6704  3251   4104   7668   40640     62367
[5, 10)       6233  3305   4033   8041   42314     63926
[10, 50)     15696  8449  11509  22145  110841    168640
[50, 100)     5672  3270   4927   9753   43464     67086
[100, 1000)  14748  9889  15869  29139  106355    176000
Grand Total: 568427


In [1]:
df_paragraph['ProductId'].value_counts().sum()

NameError: name 'df_paragraph' is not defined

# stop

In [584]:
stop

NameError: name 'stop' is not defined

In [ ]:
# next stage

In [ ]:
# this demonstrates making distance and index of a query
# k is the number of similarities to provide
# # this example is looking for the top 20 similar sentences to the first sentence in the df
start_time = time.perf_counter()
# Create a random query vector (or a set of vectors)
# dim was for the kmeans clusters
# dim = index.d  # Dimension of the vectors in the index
query_vectors = Sentence_vector_list[0].reshape(1, -1) # this reshape is required if passing in a single vector

# Perform the search
k = 20  # Number of nearest neighbors to retrieve
D, I = index.search(query_vectors, k)  # D: distances, I: indices of the neighbors
end_time = time.perf_counter()
duration = end_time - start_time
print(f"The block of code took {duration:.6f} seconds to execute.")
print("Indices of nearest neighbors: ", I)
print("Distances: ", D)

In [ ]:
# this example code runs for 105 seconds and processes 5000 vectors
start_time = time.perf_counter()
# Select the first 10 vectors from Sentence_vector_list
query_vectors = Sentence_vector_list[:1000] # same as the above example but this is provided multiple vectors as search vectors

# Search for the 5 closest vectors to each of the 10 query vectors
k = 100  # Number of nearest neighbors to find
# there was no appreciable difference in run time for how many k results to give back from 5 to 100
# it is possible that the entire similarity could be provided but the storage space cost could be problematic.
D, I = index.search(query_vectors, k)  # D: distances, I: indices of the nearest neighbors



end_time = time.perf_counter()
duration = end_time - start_time
print(f"The block of code took {duration:.6f} seconds to execute.")
print("Indices of nearest neighbors: ", I)
print("Distances: ", D)

In [ ]:
start_time = time.perf_counter()
query_vectors = Sentence_vector_list # same as the above example but this is provided multiple vectors as search vectors

# Search for the 5 closest vectors to each of the 10 query vectors
k = 100  # Number of nearest neighbors to find

D, I = index.search(query_vectors, k)  # D: distances, I: indices of the nearest neighbors



end_time = time.perf_counter()
duration = end_time - start_time
print(f"The block of code took {duration:.6f} seconds to execute.")
print("Indices of nearest neighbors: ", I)
print("Distances: ", D)

In [ ]:
stop # stop the code from running the code below

# untested concurrent code

In [ ]:

from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd

# Assuming df is your DataFrame and it contains a column 'vector' with your query vectors
query_vectors = df['vector'].tolist()  # Convert the DataFrame column to a list of vectors

# Define a function to perform the search for a single vector
def search_vector(vector):
    D, I = index.search(vector.reshape(1, -1), k)  # Reshape for single query compatibility
    return D, I

# Perform parallel search
def parallel_search(query_vectors, num_workers=4):
    results = []
    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        # Submit all the search tasks at once and return futures
        future_to_vector = {executor.submit(search_vector, vector): vector for vector in query_vectors}
        
        # As each future completes, process its result
        for future in as_completed(future_to_vector):
            vector = future_to_vector[future]
            try:
                D, I = future.result()  # Retrieves the result from the future
                results.append((D, I))
            except Exception as exc:
                print(f'Generated an exception: {exc}')
    return results

# Adjust num_workers based on your machine's capability
results = parallel_search(query_vectors, num_workers=8)


In [ ]:
stop

# examining clusters

In [ ]:
df_sentences.columns

In [ ]:
df_sentences.iloc[0]

In [ ]:
df_paragraph.columns

In [ ]:
df_paragraph.iloc[1]


In [ ]:
cluster = 12
cluster_0_df = df_paragraph[df_paragraph['Paragraph_Cluster'] == cluster]


In [ ]:
#cluster_0_df

In [ ]:
# Count the number of rows in each cluster for paragraph vectors
paragraph_cluster_counts = df_paragraph['Paragraph_Cluster'].value_counts().sort_index()
#print("Paragraph Cluster Distribution:\n", paragraph_cluster_counts)


In [ ]:
# Count the number of rows in each cluster for summary vectors
summary_cluster_counts = df_sentences['Summary_Cluster'].value_counts().sort_index()
#print("Summary Cluster Distribution:\n", summary_cluster_counts)


In [ ]:
# Count the number of rows in each cluster for sentence vectors
sentence_cluster_counts = df_sentences['Sentence_Cluster'].value_counts().sort_index()
#print("Sentence Cluster Distribution:\n", sentence_cluster_counts)


In [ ]:
import matplotlib.pyplot as plt

# Plotting the distribution of paragraph clusters
paragraph_cluster_counts.plot(kind='bar', figsize=(10, 6))
plt.title('Distribution of Paragraph Clusters')
plt.xlabel('Cluster ID')
plt.ylabel('Number of Rows')
plt.show()


In [ ]:
# Optionally, print some sample paragraphs to get a feel for the content
for paragraph in cluster_0_df['Paragraph'].sample(n=5).values:
    print(paragraph, "\n")


In [ ]:
distance_stats = cluster_0_df['Paragraph_Distance'].describe()
print(distance_stats)


In [ ]:
#pip install wordcloud

In [ ]:
from wordcloud import WordCloud

# Concatenate all paragraphs in the cluster into a single string
text = " ".join(paragraph for paragraph in cluster_0_df['Paragraph'])

# Generate a word cloud image
wordcloud = WordCloud(background_color="white").generate(text)

# Display the generated image:
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()



In [ ]:
type(wordcloud)

In [ ]:
#dir(wordcloud)


In [ ]:
print("Word frequencies:", wordcloud.words_)

In [ ]:
#print("Word layout:", wordcloud.layout_)

# Latent Dirichlet Allocation (LDA)  

In [593]:
cluster_0_df = df_paragraph[df_paragraph['Paragraph_Cluster'] == cluster]
texts = cluster_0_df['Paragraph'].values


In [592]:
from gensim import corpora
from gensim.models.ldamodel import LdaModel
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

# Ensure you have the necessary NLTK data
import nltk
nltk.download('stopwords')

# Tokenization and stop words removal
tokenizer = RegexpTokenizer(r'\w+')
stop_words = set(stopwords.words('english'))
texts_tokenized = [[word for word in tokenizer.tokenize(text.lower()) if word not in stop_words] for text in texts]

# Creating a dictionary representation of the documents
dictionary = corpora.Dictionary(texts_tokenized)

# Filtering out extremes to limit the number of features
dictionary.filter_extremes(no_below=5, no_above=0.5)

# Creating the document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts_tokenized]


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\johnny\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# LDA

In [594]:
num_topics = 50  # Adjust based on your dataset and needs
ldamodel = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)

# Print the topics
#topics = ldamodel.print_topics(num_words=4)
#for topic in topics:
#    print(topic)
# Print all 50 topics with the top 4 words in each
all_topics = ldamodel.print_topics(num_topics=50, num_words=4)
for topic in all_topics:
    print(topic)

(0, '0.097*"br" + 0.023*"bought" + 0.020*"since" + 0.018*"got"')
(1, '0.030*"great" + 0.027*"shipping" + 0.026*"br" + 0.021*"gum"')
(2, '0.103*"popcorn" + 0.021*"save" + 0.020*"great" + 0.019*"movie"')
(3, '0.023*"one" + 0.020*"red" + 0.017*"size" + 0.016*"good"')
(4, '0.038*"syrup" + 0.020*"chai" + 0.017*"good" + 0.017*"maple"')
(5, '0.023*"find" + 0.022*"asked" + 0.017*"get" + 0.016*"tasted"')
(6, '0.055*"water" + 0.050*"coconut" + 0.023*"would" + 0.018*"chips"')
(7, '0.017*"great" + 0.014*"like" + 0.012*"pretzels" + 0.012*"website"')
(8, '0.039*"years" + 0.039*"find" + 0.032*"moved" + 0.026*"live"')
(9, '0.023*"like" + 0.017*"good" + 0.015*"product" + 0.014*"amazon"')
(10, '0.035*"really" + 0.022*"product" + 0.021*"best" + 0.015*"well"')
(11, '0.067*"find" + 0.049*"amazon" + 0.038*"stores" + 0.020*"found"')
(12, '0.037*"would" + 0.026*"put" + 0.022*"think" + 0.021*"bag"')
(13, '0.039*"try" + 0.034*"tried" + 0.029*"brand" + 0.024*"brands"')
(14, '0.019*"trip" + 0.018*"old" + 0.018*"s

In [ ]:
stop

In [ ]:
stop

In [ ]:
# this experiment was to make a column in the df that showed the top 20 distances of the other

In [ ]:
# way slow This passes one vector at a time.  experiments above change this to multiple vectors for a significant savings
from tqdm.auto import tqdm

def find_top_similarities_and_distances(index, vectors, k=21):
    """
    Find the top k-1 most similar items and their distances for each vector in the given index.
    Exclude the first result to skip the query item itself.
    """
    similar_indices = []
    distances_list = []
    
    # Perform the search for each vector
    for i in tqdm(range(vectors.shape[0]), desc='Finding Similarities and Distances'):
        D, I = index.search(vectors[i:i+1], k)  # Search for the top k similar items
        similar_indices.append(I[0][1:])  # Skip the first result and keep the next 20
        distances_list.append(D[0][1:])  # Skip the distance to itself and keep the next 20
    
    return similar_indices, distances_list


In [ ]:
# way slow 128 days each
# Example for the summary vectors and index
top20_summary_indices, top20_summary_distances = find_top_similarities_and_distances(index_summary, summary_vectors, k=21)

# Update the DataFrame
df_sentences['Top20_Summary_Indices'] = top20_summary_indices
df_sentences['Top20_Summary_Distances'] = top20_summary_distances

# Repeat for sentence and paragraph vectors and indexes
top20_sentence_indices, top20_sentence_distances = find_top_similarities_and_distances(index_sentence, sentence_vectors, k=21)
df_sentences['Top20_Sentence_Indices'] = top20_sentence_indices
df_sentences['Top20_Sentence_Distances'] = top20_sentence_distances

top20_paragraph_indices, top20_paragraph_distances = find_top_similarities_and_distances(index_paragraph, paragraph_vectors, k=21)
df_paragraph['Top20_Paragraph_Indices'] = top20_paragraph_indices
df_paragraph['Top20_Paragraph_Distances'] = top20_paragraph_distances


In [ ]:
stop # some of the code below duplicates the code in the above block

In [ ]:
df_pickle_sentences_filename_clusters = os.path.join(download_dir,"amazon_reviews_pickle_sentences_20240129_clusters.pkl")

df_sentences.to_pickle(df_pickle_sentences_filename_clusters)  # Python pickle format

df_pickle_paragraph_filename_index = os.path.join(download_dir,"amazon_reviews_pickle_paragraph_20240129_index.pkl")

faiss.write_index(index_paragraph, "index_paragraph.faiss")

In [ ]:
df_pickle_sentences_filename_index = os.path.join(download_dir,"amazon_reviews_pickle_sentences_20240129_index.pkl")
faiss.write_index(index_sentence, "index_paragraph.faiss")

In [ ]:
df_pickle_paragraph_filename_clusters = os.path.join(download_dir,"amazon_reviews_pickle_paragraph_20240129_clusters.pkl")

df_paragraph.to_pickle(df_pickle_paragraph_filename_clusters)  # Python pickle format

In [ ]:
df_pickle_paragraph_filename_index = os.path.join(download_dir,"amazon_reviews_pickle_paragraph_20240129_index.pkl")

faiss.write_index(index_paragraph, "index_paragraph.faiss")

In [ ]:
df = df_sentences.iloc[0:2]

In [ ]:
df

In [ ]:
df.iloc[0]['Summary_vector']

In [ ]:
import faiss
import numpy as np

# Retrieve vectors from your DataFrame
vector1 = df.iloc[0]['Processed_vector']
vector2 = df.iloc[1]['Processed_vector']

# Ensure both vectors are numpy arrays and have the same shape
vector1 = np.array(vector1, dtype=np.float32).reshape(1, -1)
vector2 = np.array(vector2, dtype=np.float32).reshape(1, -1)

# Verify shapes are correct (this step is just for demonstration and can be removed later)
print(f"Vector1 Shape: {vector1.shape}")
print(f"Vector2 Shape: {vector2.shape}")

# Combine vectors into a single 2D array
vectors = np.vstack((vector1, vector2))

# Continue with FAISS index creation and querying
dimension = vectors.shape[1]  # Dimension of the vectors
index = faiss.IndexFlatL2(dimension)  # L2 distance for similarity measure
index.add(vectors)  # Add vectors to the index

# Perform a query with the first vector
query_vector = np.array([vector1], dtype=np.float32).reshape(1, -1)
k = 2  # Find the 2 nearest neighbors, including the query vector itself
distances, indices = index.search(query_vector, k)

# Display the results
print("Distances:", distances)
print("Indices:", indices)


In [ ]:
import faiss
import numpy as np

# Step 1: Prepare the vectors
# Assuming vector1 is the one you provided and vector2 is similarly obtained
vector1 = [1.07387170e-01,  1.54322982e-02, ...]  # Truncated for brevity
vector2 = [ ... ]  # Similar structure to vector1
vector1 = df.iloc[0]['Summary_vector']
vector2 = df.iloc[1]['Summary_vector']

vectors = np.array([vector1, vector2], dtype=np.float32)

# Step 2: Create a FAISS index
dimension = vectors.shape[1]  # Dimension of the vectors
index = faiss.IndexFlatL2(dimension)  # L2 distance for similarity measure

# Step 3: Add the vectors to the index
index.add(vectors)

# Step 4: Perform a query
# Use the first vector as a query to find its nearest neighbor
# Note: Including itself, so the nearest neighbor will be itself for k=1
query_vector = np.array([vector1], dtype=np.float32)
k = 2  # Find the 2 nearest neighbors, including the query vector itself
distances, indices = index.search(query_vector, k)

# Display the results
print("Distances:", distances)
print("Indices:", indices)


In [ ]:
df_experiment = df_sentences.head(1000).copy()  # Assuming 'df' is your original DataFrame


In [ ]:
import numpy as np

def preprocess_vectors_for_faiss(vector):
    # Ensure the vector is a numpy array of type float32 and is 2D
    if isinstance(vector, list):  # If the vector is stored as a list
        vector = np.array(vector, dtype=np.float32)
    if vector.ndim == 1:
        vector = vector.reshape(1, -1)
    return vector

# Apply this function to the 'Summary_vector' column
df_experiment['Summary_vector'] = df_experiment['Summary_vector'].apply(preprocess_vectors_for_faiss)


In [ ]:
import faiss

# Assuming you have preprocessed the vectors as 2D arrays
# Flatten the arrays for FAISS index, as it expects flat arrays
vectors = np.vstack(df_experiment['Summary_vector'].to_numpy())

# Create a FAISS index
dimension = vectors.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(vectors.astype(np.float32))  # Ensure data is float32

# Query the index (example)
query_vector = vectors[0:1]  # Use the first vector as a query example
k = 5  # Number of nearest neighbors to find
distances, indices = index.search(query_vector, k)

print(distances, indices)


In [ ]:
type(index)

In [ ]:
print(index)


In [ ]:
n_vectors = index.ntotal
vector_dim = index.d
print(f"Number of vectors: {n_vectors}, Dimensionality: {vector_dim}")


In [ ]:
is_trained = index.is_trained
print(f"Is the index trained? {is_trained}")


In [ ]:
if isinstance(index, faiss.IndexIVFFlat):
    print(f"Number of list: {index.nlist}")
    print(f"Number of probes: {index.nprobe}")


In [ ]:
# Save the index to disk
index_file = "my_index.faiss"
faiss.write_index(index, index_file)

# Load the index (for inspection or use in another program)
index_loaded = faiss.read_index(index_file)


In [ ]:
from transformers import BertTokenizer, BertModel
import torch
import numpy as np

# Initialize MinIO client and BERT model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def generate_vectors(text):
    # Check if GPU is available and use it; otherwise, use CPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Send model to device (GPU or CPU)
    model.to(device)
    model.eval()  # Ensure the model is in evaluation mode

    # Ensure no gradient calculations
    with torch.no_grad():
        # Prepare inputs and send them to the device
        inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
        
        # Forward pass, send model outputs back to CPU
        outputs = model(**inputs).last_hidden_state.mean(dim=1).squeeze().to('cpu').numpy()

    # Convert the output to float32 for compatibility and ensure it's flat
    return outputs.astype(np.float32)

# Example usage with a DataFrame
# df_sentence['Summary_vector'] = df_sentence['Summary'].progress_apply(lambda x: generate_vectors(x) if isinstance(x, str) else np.nan)


# junk end

In [ ]:
vectors = np.stack(df_sentences['Summary_vector'].values).astype(np.float32)

In [ ]:
vectors = df_sentences['Summary_vector'].values

In [ ]:
type(vectors[0])

In [ ]:
vectors = df_sentences['Summary_vector'].values.astype(np.float32)  # Assuming vectors are stored as numpy arrays


In [ ]:
type(vectors)

In [ ]:
stop

In [ ]:
import chromadb

# Initialize the Chroma client with a specified database path
db_path = download_dir  # Specify the path where you want to store the ChromaDB
#chroma_client = chromadb.Client(path=db_path) # this gave an error
#client = chromadb.PersistentClient(path="/path/to/save/to") # from web site
chroma_client = chromadb.PersistentClient(path=db_path)

In [ ]:
def separate_metadata_and_vectors(df):
    vector_columns = [col for col in df.columns if 'vector' in col.lower()]
    metadata_columns = [col for col in df.columns if 'vector' not in col.lower()]
    return metadata_columns, vector_columns


In [ ]:
def load_df_to_chromadb(df, collection_name, text_columns, embedding_columns, db_path, ignore_columns=None):
    if ignore_columns is None:
        ignore_columns = []

    chroma_client = chromadb.PersistentClient(path=db_path)

    try:
        collection = chroma_client.get_collection(collection_name)
    except Exception as e:
        print(f"Collection {collection_name} not found, creating a new one. Error: {e}")
        collection = chroma_client.create_collection(name=collection_name)

    #for _, row in df.iterrows():
    for _, row in tqdm(df.iterrows(), total=len(df), desc=f"Loading to {collection_name}"):
        metadata = {col: row[col] for col in row.index if col not in embedding_columns + text_columns + ignore_columns}
        #embeddings = [row[col] for col in embedding_columns if col in row and col not in ignore_columns]

        # Ensure embeddings are converted to list if they are numpy arrays
        embeddings = [row[col].tolist() for col in embedding_columns if col in row and col not in ignore_columns and isinstance(row[col], np.ndarray)][0]
        embeddings = embeddings[0] if embeddings else None
        if embeddings is not None:
            embeddings_list.append(embeddings)
        
        documents = [row[col] for col in text_columns if col in row]
        
        document_id = f"{collection_name}_{row['Id']}"
        if 'P_index' in row and 'S_sentence_number' in row:
            document_id += f"_{row['P_index']}_{row['S_sentence_number']}"

        collection.add(
            documents=documents,
            embeddings=embeddings,  # Each embedding is added separately
            metadatas=[metadata],
            ids=[document_id]
        )

In [ ]:
def delete_collection(collection):
    try:
        chroma_client.delete_collection(collection)
        print("Collection '{collection}' deleted successfully.")
    except Exception as e:
        print(f"Failed to delete collection: {e}")

In [ ]:
delete_collection('sentences_collection')

In [ ]:
# Create collections

delete_collection('sentences_collection')
sentences_collection = chroma_client.create_collection(name="sentences_collection")


delete_collection('paragraphs_collection')
paragraphs_collection = chroma_client.create_collection(name="paragraphs_collection")


delete_collection('paragraphs_collection')
summary_collection = chroma_client.create_collection(name="paragraphs_collection")


In [ ]:
#type(df_sentences.iloc[0]['Sentence_vector'].tolist())

In [ ]:
#df_sentences.iloc[0]['Sentence_vector'][0].tolist()